<div class="alert" style="background-color:#006400; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:15px 15px; color:#FFFFFF; font-size:32px'>HyperTuning for 180 Datasets</h1></div>

The work is under the **"Master Thesis"** by **Chau Tran** with the supervision from **Prof. Roland Olsson**.

## 1. Packages and Datasets

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, MaxPooling2D,AveragePooling2D,GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten, Dropout, Reshape, BatchNormalization, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorboard
import kerastuner as kt #(kt.tuners.RandomSearch, kt.tuners.Hyperband)
from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler

from functools import partial
from matplotlib import rc, style
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
from scipy import stats

import sys, os, math, time, datetime

print("kt: ", kt.__version__)
print("tf: ", tf.__version__)
print(os.getcwd())

%matplotlib inline
%config InlineBackend.figure_format='retina'

style.use("seaborn")
pd.plotting.register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale = 1)

# rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

tf.debugging.set_log_device_placement(False)

strategy = tf.distribute.MirroredStrategy()

def seperateValues(data):
    x_data, y_data = None, None
    for i in range(data.shape[0]):
        x_data_i = data[i].reshape(-1, noInput+noOutput).astype('float32')
        x_data_i, y_data_i = x_data_i[:, 0:noInput], x_data_i[-1, noInput:]
        x_data = x_data_i[np.newaxis,:,:] if x_data is None else np.append(x_data, x_data_i[np.newaxis,:,:], axis=0)
        y_data = y_data_i.reshape(1, -1) if y_data is None else np.append(y_data, y_data_i.reshape(1, -1), axis=0)
    return x_data, y_data

path = './Version9.128timesteps'
fileslist = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
val_performance, train_time = {}, {}
FILESNUMBER = 10
LSTMNUMBER  = 3
loss_method = 'binary_crossentropy'
for i in range(FILESNUMBER):
    # Getting data from csv file
    filepath = join(path,fileslist[i])
    with open(filepath, "r") as fp:
        [noInput, noOutput] = [int(x) for x in fp.readline().split(',')]
    rdf = np.array(pd.read_csv(filepath, skiprows=1))

    print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
    df_val, df_train = train_test_split(rdf,test_size=0.5)
    print(df_train.shape, df_val.shape)

    print('Step 2: Separating values and labels.')
    x_train, y_train = seperateValues(df_train)
    x_val, y_val = seperateValues(df_val)
    print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
    print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)
    
    print('Step 3: LSTM modeling.')
    lstm_model = tf.keras.Sequential()
    lstm_model.add(
      tf.keras.layers.LSTM(
          units=128, 
          input_shape=[x_train.shape[1], x_train.shape[2]],
          activation='tanh', recurrent_activation='sigmoid',
          unroll =False,
          use_bias=True,
          recurrent_dropout=0,
          return_sequences=False
      )
    )
    lstm_model.add(tf.keras.layers.Dense(y_train.shape[1], activation='tanh'))
    adam = tf.optimizers.Adam(learning_rate = 0.001, decay=1e-6)
    lstm_model.compile(loss=loss_method, optimizer=adam, metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.51)])
    val_performance[fileslist[i].split('.')[6]] = []
    train_time[fileslist[i].split('.')[6]] = []
    for lstm_no in range(LSTMNUMBER):
        strt_time = datetime.datetime.now() 
        lstm_model.fit(
            x_train, y_train,
            epochs=8,
            batch_size=1,
            verbose=0,
            validation_data=(x_val, y_val),
            shuffle=True,
            use_multiprocessing=True
        )
        curr_time = datetime.datetime.now()
        display(lstm_model.summary())
        timedelta = curr_time - strt_time
        train_time[fileslist[i].split('.')[6]].append(timedelta.total_seconds())
        val_performance[fileslist[i].split('.')[6]].append(lstm_model.evaluate(x_val, y_val))
df = (pd.DataFrame.from_dict(val_performance, orient='index'))
df[0] = df.apply(lambda x: round(x[0][1], 5),axis=1)
df[1] = df.apply(lambda x: round(x[1][1], 5),axis=1)
df[2] = df.apply(lambda x: round(x[2][1], 5),axis=1)
df = pd.concat([df, pd.DataFrame.from_dict(train_time, orient='index').rename(columns = {0: 'Training_Time'})], axis=1)
df = df.rename(columns={0: '1st_run', 1: '2nd_run', 2: '3rd_run'})
df.to_csv('./Results/{}_10datasets.csv'.format(loss_method), index=True, index_label='Datasets')

kt:  1.0.2
tf:  2.6.0
/home/ifeai/ChauTran/git/0_HIOF_Studying/0_MasterProject
Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(5000, 1024) (4999, 1024)
Step 2: Separating values and labels.
+ Training set:    (5000, 128, 3) (5000, 5) float32
+ Validating set:  (4999, 128, 3) (4999, 5) float32
Step 3: LSTM modeling.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               67584     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


None

157/157 [==============================] - 1s 3ms/step - loss: 0.7318 - binary_accuracy: 0.9522
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               67584     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


None

157/157 [==============================] - 1s 5ms/step - loss: 0.7282 - binary_accuracy: 0.9528
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               67584     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


...

157/157 [==============================] - 1s 5ms/step - loss: 0.3408 - binary_accuracy: 0.8540
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               67584     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


None

157/157 [==============================] - 1s 4ms/step - loss: 0.0932 - binary_accuracy: 0.9904
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               67584     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


None

157/157 [==============================] - 1s 4ms/step - loss: 0.0105 - binary_accuracy: 0.9990
Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(5000, 1024) (4999, 1024)
Step 2: Separating values and labels.
+ Training set:    (5000, 128, 3) (5000, 5) float32
+ Validating set:  (4999, 128, 3) (4999, 5) float32
Step 3: LSTM modeling.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, MaxPooling2D,AveragePooling2D,GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten, Dropout, Reshape, BatchNormalization, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorboard
import kerastuner as kt #(kt.tuners.RandomSearch, kt.tuners.Hyperband)
from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler

from functools import partial
from matplotlib import rc, style
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
from scipy import stats

import sys, os, math, time, datetime

print("kt: ", kt.__version__)
print("tf: ", tf.__version__)
print(os.getcwd())

%matplotlib inline
%config InlineBackend.figure_format='retina'

style.use("seaborn")
pd.plotting.register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale = 1)

# rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

tf.debugging.set_log_device_placement(False)

strategy = tf.distribute.MirroredStrategy()

def seperateValues(data):
    x_data, y_data = None, None
    for i in range(data.shape[0]):
        x_data_i = data[i].reshape(-1, noInput+noOutput).astype('float32')
        x_data_i, y_data_i = x_data_i[:, 0:noInput], x_data_i[-1, noInput:]
        x_data = x_data_i[np.newaxis,:,:] if x_data is None else np.append(x_data, x_data_i[np.newaxis,:,:], axis=0)
        y_data = y_data_i.reshape(1, -1) if y_data is None else np.append(y_data, y_data_i.reshape(1, -1), axis=0)
    return x_data, y_data

snapshot = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
path = './Version9.128timesteps'
fileslist = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
val_performance, train_time = {}, {}
FILESNUMBER = 1
LSTMNUMBER  = 1
loss_method = 'mse'
for i in range(FILESNUMBER):
    # Getting data from csv file
    filepath = os.path.join(path,fileslist[i])
    with open(filepath, "r") as fp:
        [noInput, noOutput] = [int(x) for x in fp.readline().split(',')]
    rdf = np.array(pd.read_csv(filepath, skiprows=1))

    print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
    df_val, df_train = train_test_split(rdf,test_size=0.5)
    print(df_train.shape, df_val.shape)

    print('Step 2: Separating values and labels.')
    x_train, y_train = seperateValues(df_train)
    x_val, y_val = seperateValues(df_val)
    print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
    print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)
    
    print('Step 3: Defining a LSTM model for tuning.')
    log_dir_lstm_v1 = "logs//hparams//" + snapshot
    lstm_af = ['relu', 'sigmoid', 'tanh', 'softmax', 'softsign', 'selu', 'elu']
    lstm_raf = ['sigmoid', 'softmax']
    dense_af = ['tanh', 'sigmoid', 'softmax']
    loss_methods = ['mse', 'binary_crossentropy']
    thresholds = [0.5, 0.51, 0.6, 0.7]
    def tunner_lstm_model_v1(hp):
        """Builds a recurrent model."""
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(x_train.shape[1], x_train.shape[2])))
        model.add(tf.keras.layers.LSTM(units=8, 
                       activation=hp.Choice('af_LSTM', lstm_af),
                       recurrent_activation=hp.Choice('raf_LSTM', lstm_raf),
                       unroll =False,
                       use_bias=True,
                       recurrent_dropout=0,
                       return_sequences=False))

        model.add(tf.keras.layers.Dense(y_train.shape[1], hp.Choice('raf_LSTM', dense_af)))
        if (hp.Choice('optimizer', ['adam', 'sgd']) == 'adam'):
            optimizer = tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
        else:
            optimizer = tf.keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
        model.compile(optimizer=optimizer,
                      loss=hp.Choice('loss_methods', loss_methods),
                      metrics=[tf.keras.metrics.BinaryAccuracy(threshold=hp.Choice('thresholds_BA', thresholds)),
                               tf.keras.metrics.Precision(name='precision'),
                               tf.keras.metrics.Recall(name='recall')
                              ])
        return model

    lstm_tuner_v1=kt.tuners.Hyperband(
        tunner_lstm_model_v1,
        objective=kt.Objective('val_binary_accuracy', direction='max'),
        max_epochs=8,
        seed=42,
        directory="logs/tuner/",
        project_name="MasterProject",
        overwrite=True,
        logger=TensorBoardLogger(
            metrics=['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy', 'precision', 'recall'], logdir=log_dir_lstm_v1,
        ) # add only this argument
    )

    setup_tb(lstm_tuner_v1)  # (Optional) For more accurate visualization.
    lstm_tuner_v1.search(x_train, y_train,
                 epochs=8,
                 batch_size=1,
                 validation_data=(x_val, y_val),
                 shuffle=False
                )
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir_lstm_v1, profile_batch = '500,520')
    bestparams_v1 = lstm_tuner_v1.get_best_hyperparameters(1)[0]
    hyper_model_v1 = lstm_tuner_v1.hypermodel.build(bestparams_v1)
    
    val_performance[fileslist[i].split('.')[6]] = []
    train_time[fileslist[i].split('.')[6]] = []
    strt_time = datetime.datetime.now() 
    training_history_v1 = hyper_model_v1.fit(
        X_train, # input
        y_train, # output
        batch_size=1,
        verbose=0, # Suppress chatty output; use Tensorboard instead
        epochs=8,
        validation_data=(x_val, y_val),
        callbacks=[tensorboard_callback])
    curr_time = datetime.datetime.now()
    print("Average loss: ", np.average(training_history_v1.history['loss']))
    print("Average val loss: ", np.average(training_history_v1.history['val_loss']))
    print("Average accuracy: ", np.average(training_history_v1.history['binary_accuracy']))
    print("Average val accuracy: ", np.average(training_history_v1.history['val_binary_accuracy']))
    timedelta = curr_time - strt_time
    train_time[fileslist[i].split('.')[6]] = timedelta.total_seconds()
    val_performance[fileslist[i].split('.')[6]] = hyper_model_v1.evaluate(x_val, y_val)
    
    
df = (pd.DataFrame.from_dict(val_performance, orient='index'))
df[0] = df.apply(lambda x: round(x[0][1], 5),axis=1)
df[1] = df.apply(lambda x: round(x[1][1], 5),axis=1)
df[2] = df.apply(lambda x: round(x[2][1], 5),axis=1)
time_df = pd.DataFrame.from_dict(train_time, orient='index').rename(columns = {0: 'Training_Time'})
time_df['Training_Time'] = time_df.apply(lambda x: round(sum(x) / len(x), 5), axis=1)
df = pd.concat([df, time_df], axis=1)
df = df.rename(columns={0: '1st_run', 1: '2nd_run', 2: '3rd_run'})
df.to_csv('./Results/{}_10datasets.csv'.format(loss_method), index=True, index_label='Datasets')

Trial 2 Complete [00h 13m 11s]
val_binary_accuracy: 0.6619059443473816

Best val_binary_accuracy So Far: 0.6619059443473816
Total elapsed time: 00h 26m 26s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
af_LSTM           |sigmoid           |softsign          
raf_LSTM          |sigmoid           |sigmoid           
optimizer         |adam              |sgd               
learning_rate     |0.01              |0.01              
loss_methods      |mse               |binary_crossent...
thresholds_BA     |0.51              |0.51              
tuner/epochs      |3                 |3                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |1                 |1                 
tuner/round       |0                 |0                 

Epoch 1/3
4999/5000 [============================>.] - ETA: 0s - loss: 0.1401 - binary_accuracy: 0.7585 - precision: 0.7310 - recall: 0.7202